# Approaches to modeling and simulation

`Simulate.jl` supports different approaches to modeling and simulation of *discrete event systems (DES)*. It provides three major schemes: 1) an event-scheduling scheme, 2) a process-oriented scheme and 3) continuous sampling. With them different modeling strategies can be applied.

A problem can be expressed differently through various modeling approaches. A simple problem can illustrate this :

> A server *takes* something from an input, *processes* it for some time and
> *puts* it out to an output. There are 8 servers in the system, 4 foos and 4 bars
> interacting with each other via two channels.


## Event based modeling

In this view *events* occur in time and trigger further events. Here the three server actions are seen as events and can be described in an event graph:

![event graph](../src/images/event.png)

You define a data structure for the server, provide functions for the three actions, create channels and servers and start:


In [1]:
using Simulate, Printf, Random

mutable struct Server1
  id::Int64
  name::AbstractString
  input::Channel
  output::Channel
  op     # operation to take
  token  # current token

  Server1(id, name, input, output, op) = new(id, name, input, output, op, nothing)
end

function take(S::Server1)
    if isready(S.input)
        S.token = take!(S.input)
        @printf("%5.2f: %s %d took token %d\n", tau(), S.name, S.id, S.token)
        event!(SF(put, S), after, rand())         # call put after some time
    else
        event!(SF(take, S), SF(isready, S.input)) # call again if input is ready
    end
end

function put(S::Server1)
    put!(S.output, S.op(S.id, S.token))
    S.token = nothing
    take(S)
end

reset!(𝐶)
Random.seed!(123)

ch1 = Channel(32)  # create two channels
ch2 = Channel(32)

s = shuffle(1:8)
for i in 1:2:8
    take(Server1(s[i], "foo", ch1, ch2, +))
    take(Server1(s[i+1], "bar", ch2, ch1, *))
end

put!(ch1, 1) # put first token into channel 1
run!(𝐶, 10)

 0.01: foo 4 took token 1
 0.12: bar 6 took token 5
 0.29: foo 1 took token 30
 0.77: bar 8 took token 31
 1.64: foo 2 took token 248
 2.26: bar 3 took token 250
 2.55: foo 7 took token 750
 3.02: bar 5 took token 757
 3.30: foo 4 took token 3785
 3.75: bar 6 took token 3789
 4.34: foo 1 took token 22734
 4.60: bar 8 took token 22735
 5.31: foo 2 took token 181880
 5.61: bar 3 took token 181882
 5.90: foo 7 took token 545646
 6.70: bar 5 took token 545653
 6.91: foo 4 took token 2728265
 7.83: bar 6 took token 2728269
 8.45: foo 1 took token 16369614
 9.26: bar 8 took token 16369615
 9.82: foo 2 took token 130956920


"run! finished with 20 clock events, 1000 sample steps, simulation time: 10.0"

## State based modeling

Here the server has three states: *Idle*, *Busy* and *End* (where *End* does nothing). On an arrival event it resets its internal clock $x=0$ and determines the service time $t_s$, moves to *Busy*, *works* on its input and puts it out when $t_s$ is over. Then it goes back to *Idle*. A state transition diagram (Mealy model) of the timed automaton would look like:

![timed automaton](../src/images/state.png)

Again you need a data structure for the server (state …). You define states and events and implement a $δ$ transition function with two methods. Thereby you dispatch on states and events. Since you don't need to implement all combinations of states and events, you may implement a fallback transition.

In [2]:
abstract type Q end  # states
struct Idle <: Q end
struct Busy <: Q end
abstract type Σ end  # events
struct Arrive <: Σ end
struct Leave <: Σ end

mutable struct Server2
    id::Int64
    name::AbstractString
    input::Channel
    output::Channel
    op     # operation to take
    state::Q
    token  # current token

    Server2(id, name, input, output, op) = new(id, name, input, output, op, Idle(), nothing)
end

arrive(A) = event!(SF(δ, A, A.state, Arrive()), SF(isready, A.input))

function δ(A::Server2, ::Idle, ::Arrive)
    A.token = take!(A.input)
    @printf("%5.2f: %s %d took token %d\n", tau(), A.name, A.id, A.token)
    A.state=Busy()
    event!(SF(δ, A, A.state, Leave()), after, rand())
end

function δ(A::Server2, ::Busy, ::Leave)
    put!(A.output, A.op(A.id,A.token))
    A.state=Idle()
    arrive(A)
end

δ(A::Server1, q::Q, σ::Σ) =               # fallback transition
        println(stderr, "$(A.name) $(A.id) undefined transition $q, $σ")

reset!(𝐶)
Random.seed!(123)

ch1 = Channel(32)  # create two channels
ch2 = Channel(32)

s = shuffle(1:8)
for i in 1:2:8
    arrive(Server2(s[i], "foo", ch1, ch2, +))
    arrive(Server2(s[i+1], "bar", ch2, ch1, *))
end

put!(ch1, 1) # put first token into channel 1
run!(𝐶, 10)

 0.01: foo 4 took token 1
 0.12: bar 6 took token 5
 0.29: foo 1 took token 30
 0.77: bar 8 took token 31
 1.64: foo 2 took token 248
 2.26: bar 3 took token 250
 2.55: foo 7 took token 750
 3.02: bar 5 took token 757
 3.30: foo 4 took token 3785
 3.75: bar 6 took token 3789
 4.34: foo 1 took token 22734
 4.60: bar 8 took token 22735
 5.31: foo 2 took token 181880
 5.61: bar 3 took token 181882
 5.90: foo 7 took token 545646
 6.70: bar 5 took token 545653
 6.91: foo 4 took token 2728265
 7.83: bar 6 took token 2728269
 8.45: foo 1 took token 16369614
 9.26: bar 8 took token 16369615
 9.82: foo 2 took token 130956920


"run! finished with 20 clock events, 1000 sample steps, simulation time: 10.0"

## Activity based modeling

The server's *activity* is the processing of the token. A timed Petri net would look like:

![timed petri net](../src/images/activity.png)

The *arrive* "transition" puts a "token" in the *Queue*. If both "places" *Idle* and *Queue* have tokens, the server *takes* them, shifts one to *Busy* and *puts* out two after a timed transition with delay $v_{put}$. Then it is *Idle* again and the cycle restarts.

The server's activity is described by the blue box. Following the Petri net, you should implement a state variable with states Idle and Busy, but you don't need to if you separate the activities in time. You need a data structure for the server and define a function for the activity:

In [3]:
mutable struct Server3
  id::Int64
  name::AbstractString
  input::Channel
  output::Channel
  op     # operation
  token  # current token

  Server3(id, name, input, output, op) = new(id, name, input, output, op, nothing)
end

arrive(S::Server3) = event!(SF(serve, S), SF(isready, S.input))

function serve(S::Server3)
    S.token = take!(S.input)
    @printf("%5.2f: %s %d took token %d\n", tau(), S.name, S.id, S.token)
    event!((SF(put!, S.output, S.op(S.id, S.token)), SF(arrive, S)), after, rand())
end

reset!(𝐶)
Random.seed!(123)

ch1 = Channel(32)  # create two channels
ch2 = Channel(32)

s = shuffle(1:8)
for i in 1:2:8
    arrive(Server3(s[i], "foo", ch1, ch2, +))
    arrive(Server3(s[i+1], "bar", ch2, ch1, *))
end

put!(ch1, 1) # put first token into channel 1
run!(𝐶, 10)

 0.01: foo 4 took token 1
 0.12: bar 6 took token 5
 0.29: foo 1 took token 30
 0.77: bar 8 took token 31
 1.64: foo 2 took token 248
 2.26: bar 3 took token 250
 2.55: foo 7 took token 750
 3.02: bar 5 took token 757
 3.30: foo 4 took token 3785
 3.75: bar 6 took token 3789
 4.34: foo 1 took token 22734
 4.60: bar 8 took token 22735
 5.31: foo 2 took token 181880
 5.61: bar 3 took token 181882
 5.90: foo 7 took token 545646
 6.70: bar 5 took token 545653
 6.91: foo 4 took token 2728265
 7.83: bar 6 took token 2728269
 8.45: foo 1 took token 16369614
 9.26: bar 8 took token 16369615
 9.82: foo 2 took token 130956920


"run! finished with 20 clock events, 1000 sample steps, simulation time: 10.0"

## Process based modeling

Here you combine it all in a simple function of *take!*-*delay!*-*put!* like in the activity based example, but running in the loop of a process. Processes can wait or delay and are suspended and reactivated by Julia's scheduler according to background events. There is no need to handle events explicitly and no need for a server data type since a process keeps its own data. Processes must look careful to their timing and therefore you must enclose the IO-operation in a `now!` call:

In [5]:
function simple(input::Channel, output::Channel, name, id, op)
    token = take!(input)         # take something, eventually wait for it
    now!(SF(println, @sprintf("%5.2f: %s %d took token %d", tau(), name, id, token)))
    d = delay!(rand())           # wait for a given time
    put!(output, op(token, id))  # put something else out, eventually wait
end

ch1 = Channel(32)  # create two channels
ch2 = Channel(32)

for i in 1:2:8    # create and register 8 SimProcesses
    process!(SP(i, simple, ch1, ch2, "foo", i, +))
    process!(SP(i+1, simple, ch2, ch1, "bar", i+1, *))
end

reset!(𝐶)
put!(ch1, 1) # put first token into channel 1
run!(𝐶, 10)

 0.00: foo 1 took token 1
 0.79: bar 2 took token 2
 1.15: foo 3 took token 4
 2.05: bar 4 took token 7
 2.58: foo 5 took token 28
 2.61: bar 6 took token 33
 3.51: foo 7 took token 198
 4.45: bar 8 took token 205
 5.07: foo 1 took token 1640
 5.42: bar 2 took token 1641
 5.99: foo 3 took token 3282
 6.19: bar 4 took token 3285
 6.57: foo 5 took token 13140
 7.33: bar 6 took token 13145
 7.52: foo 7 took token 78870
 7.75: bar 8 took token 78877
 7.85: foo 1 took token 631016
 8.48: bar 2 took token 631017
 9.43: foo 3 took token 1262034
 9.97: bar 4 took token 1262037


"run! finished with 39 clock events, 0 sample steps, simulation time: 10.0"

## Comparison

The output of the last example is different from the first three approaches because we did not shuffle (the shuffling of the processes is done by the scheduler). So if the output depends very much on the sequence of events and you need to have reproducible results, explicitly controlling for the events like in the first three examples is preferable. If you are more interested in statistical evaluation - which is often the case -, the 4th approach is appropriate.

All four approaches can be expressed in `Simulate.jl`. Process based modeling seems to be the simplest and the most intuitive approach, while the first three are more complicated. But they are also more structured and controllable , which comes in handy for more complicated examples. After all, parallel processes are often tricky to control and to debug. But you can combine the approaches and take the best from all worlds.

## Combined approach

Physical systems can be modeled as *continuous systems* (nature does not jump), *discrete systems* (nature jumps here!) or *hybrid systems* (nature jumps sometimes).

While continuous systems are the domain of differential equations, discrete and hybrid systems may be modeled easier with `Simulate.jl` by combining the *event-scheduling*, the *process-based* and the *continuous-sampling* schemes.

### A hybrid system

(empty)

## Theories

There are some theories about the different approaches (1) event based, (2) state based, (3) activity based and (4) process based. Choi and Kang [1] have written an entire book about the first three approaches. Basically they can be converted to each other. Cassandras and Lafortune [2] call those "the event scheduling scheme" and the 4th approach "the process-oriented simulation scheme" [3]. There are communities behind the various views and `Simulate.jl` wants to be useful for them all.

[1]:  [Choi and Kang: *Modeling and Simulation of Discrete-Event Systems*, Wiley, 2013](https://books.google.com/books?id=0QpwAAAAQBAJ)

[2]: [Cassandras and Lafortune: *Introduction to Discrete Event Systems*, Springer, 2008, Ch. 10](https://books.google.com/books?id=AxguNHDtO7MC)

[3]: to be fair, the 4th approach is called by Choi and Kang "parallel simulation".